In [1]:
import torch
from torch import nn, optim

import torch.nn.functional as F
from einops import rearrange
from typing import Optional
from ember import (
    Transformer,
    MultiHeadLatentAttn,
    Tokenizer,
    RoPE,
    apply_rotary_pos_emb,
)

/Users/ryanpegoud/Documents/Projects/ember/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
class GroupedQueryAttn(nn.Module):
    """
    Grouped Query Attention.

    Einsum notation:
        - `B`: batch size
        - `S`: sequence length
        - `NH`: number of heads
        - `HD`: head dimension
    """

    def __init__(
        self,
        model_dim: int,
        n_query_heads: int,
        n_query_groups: int,
        rope_theta: Optional[int] = 50_000,
    ):
        super().__init__()
        assert (
            model_dim % n_query_heads == 0
        ), f"{model_dim=} is not divisible by {n_query_heads=}"
        assert (
            n_query_heads % n_query_groups == 0
        ), f"{n_query_heads=} is not divisible by {n_query_groups=}"

        self.model_dim = model_dim
        self.n_query_heads = n_query_heads
        self.n_query_groups = n_query_groups
        self.kv_heads_per_q_head = n_query_heads // n_query_groups

        self.q_head_dim = model_dim // n_query_heads
        self.kv_head_dim = model_dim // self.kv_heads_per_q_head
        self.kv_single_head_dim = self.kv_head_dim // self.n_query_groups

        self.fused_qkv = nn.Linear(model_dim, model_dim + self.kv_head_dim * 2)
        self.out_proj = nn.Linear(model_dim, model_dim)

        self.rope = RoPE(dim=self.kv_single_head_dim, base=rope_theta)

    def duplicate_heads(self, x: torch.Tensor) -> torch.Tensor:
        """
        Creates duplicate views of shared k/v heads to align
        with queries shape. The created views hold no space in memory.
        """
        x = x.unsqueeze(2).expand(-1, -1, self.kv_heads_per_q_head, -1, -1)
        x = x.reshape(x.shape[0], -1, *x.shape[3:])
        return x

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        fused_proj = self.fused_qkv(x)

        q, k, v = torch.split(
            fused_proj, [self.model_dim, self.kv_head_dim, self.kv_head_dim], dim=-1
        )

        # split q, k, v in different heads
        q = rearrange(q, "B S (NH HD) -> B NH S HD", NH=self.n_query_heads)
        k, v = map(
            lambda x: rearrange(x, "B S (NH HD) -> B NH S HD", NH=self.n_query_groups),
            (k, v),
        )

        cos, sin = self.rope(q)
        cos, sin = map(lambda x: x.transpose(0, 2), (cos, sin))
        q, k = apply_rotary_pos_emb(q, k, cos, sin)

        # duplicate shared heads to align q, k, v shapes
        k, v = map(self.duplicate_heads, (k, v))

        attn_outputs = F.scaled_dot_product_attention(q, k, v, is_causal=True)
        attn_outputs = rearrange(attn_outputs, "B NH S HD -> B S (NH HD)")

        return self.out_proj(attn_outputs)

In [25]:
B, S, D = 8, 1024, 512
mla_kwargs = {
    "latent_dim": 256,
    "pos_dim": 128,
    "n_heads": 16,
}
gqa_kwargs = {
    "n_query_heads": 8,
    "n_query_groups": 4,
}

tk = Tokenizer()

mini_llama = Transformer(
    vocab_size=tk.vocab_size,
    model_dim=D,
    hidden_dim=1024,
    attn_module=GroupedQueryAttn,
    attn_kwargs=gqa_kwargs,
    n_attn_blocks=3,
)

# mini_deepseek = Transformer(
#     vocab_size=tk.vocab_size,
#     model_dim=D,
#     hidden_dim=1024,
#     attn_module=MultiHeadLatentAttn,
#     attn_kwargs=mla_kwargs,
#     n_attn_blocks=3,
# )

<bound method Module.parameters of Transformer(
  (embed): Embedding(128256, 512)
  (attn_blocks): ModuleList(
    (0-2): 3 x AttentionBlock(
      (mlp): SwiGLU(
        (W): Linear(in_features=512, out_features=682, bias=False)
        (V): Linear(in_features=512, out_features=682, bias=False)
        (W2): Linear(in_features=682, out_features=512, bias=False)
      )
      (norm): RMSNorm()
      (attn): GroupedQueryAttn(
        (fused_qkv): Linear(in_features=512, out_features=1024, bias=True)
        (out_proj): Linear(in_features=512, out_features=512, bias=True)
        (rope): RoPE()
      )
    )
  )
  (norm): RMSNorm()
)>
Parameter count: 7.12e+07


In [26]:
txt = ["Hey there", "How are you?"]
x = tk(txt)
x.shape

with torch.no_grad():
    out_l = mini_llama(x)
    print(out_l.shape)
    # out_d = mini_deepseek(x)
    # print(out_d.shape)

torch.Size([2, 5, 128256])
